In [1]:
print('Hello world')

Hello world


In [15]:
import numpy as np
import matplotlib as plt

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [16]:
%setup_logging

Global logger already initialized!


In [17]:
%cuda_context_handler context

Registering context in user workspace
Context already registered! Ignoring


In [25]:
kernel_src = """

__global__ void shmemReduction(float* output, float* input, int size) {
    //First we stride through global memory and compute
    //the maximum for every thread
    int gid = blockIdx.x * blockDim.x + threadIdx.x; //blockIdx is always 0 because we use just 1 block
    
    float max_value = -9999999.99;    
    for (int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    
    //Temporary write to memory to check if things work so far
    output[threadIdx.x] = max_value;
    
    //Store the per-thread maximum in shared memory
    __shared__ float max_shared[128];
    max_shared[threadIdx.x] = max_value;
        
    //Synchronize so that all thread see the same shared memory
    __syncthreads();
    
    //Find the maximum in shared memory
    
    //Reduce from 128 to 64
     if (threadIdx.x < 64) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 64]);
    }
    
    //
    //
    __syncthreads();    
    
    //Reduce from 64 to 32
     if (threadIdx.x < 32) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 32]);
    }
    
    //Reduce from 32 to 16
     if (threadIdx.x < 16) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 16]);
    }
    
    //Reduce from 16 to 8
     if (threadIdx.x < 8) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 8]);
    }
    
    //Reduce from 8 to 4
     if (threadIdx.x < 4) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 4]);
    }
    
    //Reduce from 4 to 2
     if (threadIdx.x < 2) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 2]);
    }
    
    //Reduce from 2 to 1
     if (threadIdx.x < 1) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x + 1]);
    }
    
    //Finally write out to output
    if (threadIdx.x == 0) {
        output[0] = max_shared[0];    
    }

}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(7): warning: variable "gid" was declared but never referenced




In [22]:
n = 128
a = np.random.random((1,n)).astype(np.float32)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 64
b = np.empty((1,num_threads), dtype=np.float32)

b_g = GPUArray(b.shape, b.dtype)

In [24]:
block_size = (num_threads, 1, 1)
grid_size = (1, 1, 1)

kernel_function(b_g, a_g, np.int32(n), np.int32(n),grid=grid_size, block=block_size)

b_g.get(b)

print(a)
print(b)

[[1.73013583e-01 8.73201132e-01 7.15976775e-01 4.29855943e-01
  7.98393726e-01 6.90453649e-01 4.65036966e-02 2.94540346e-01
  9.56288040e-01 2.96723366e-01 8.00974071e-01 9.98879850e-01
  7.92831182e-01 8.63308728e-01 6.15976274e-01 1.59680218e-01
  3.05530250e-01 6.17582798e-01 5.25161922e-01 9.84551534e-02
  9.36919093e-01 3.47840071e-01 5.93698800e-01 5.15059352e-01
  2.80975908e-01 1.70232698e-01 4.40574974e-01 8.81036557e-03
  9.07207072e-01 8.45832229e-01 4.25834179e-01 1.77027866e-01
  2.78126299e-01 3.65826905e-01 1.28332987e-01 5.31588674e-01
  8.48873734e-01 1.07728995e-01 2.94347793e-01 3.21659476e-01
  5.69149137e-01 5.14385998e-01 6.74198789e-04 1.25035644e-01
  4.53276306e-01 1.29071116e-01 4.30171847e-01 6.19124591e-01
  2.83433765e-01 1.28407419e-01 9.46896896e-02 1.78152412e-01
  2.22793013e-01 2.03935713e-01 2.90741563e-01 4.49092180e-01
  2.20332116e-01 4.68699694e-01 1.12560056e-02 9.14939702e-01
  7.22574472e-01 7.14568138e-01 5.39393663e-01 2.61559874e-01]]
[[0.99